In [3]:
import os
import torch as pyTorch
import torch.nn as nn
import torch.nn.functional as function
import torch.optim as optim
import numpy as np

In [4]:
# Deep Q Network
class DeepQNetwork(nn.Module):
    def __init__(self, learning_rate, input_dimensions, fc1_dimensions, fc2_dimensions, num_actions):
        super(DeepQNetwork, self).__init__()
        self.input_dimensions
        self.fc1_dimensions = fc1_dimensions
        self.fc2_dimensions = fc2_dimensions
        self.num_actions = num_actions
        self.fc1 = nn.Linear(*self.input_dim, self.fc1_dimensions)
        self.fc2 = nn.Linear(self.fc1_dimensions, self.fc2_dimensions)
        self.fc3 = nn.Linear(self.fc2_dimensions, self.num_actions)
        self.optimizer = optim.Adam(self.parameters(), lr = learning_rate)
        self.loss = nn.MSELoss()
        
        #CPU usage
        self.device = pyTorch.device()
    
    def forward(self, observation):
        # For this Open AI Gym environment, we don't have to do flattening
        state = pyTorch.Tensor(observation).to(self.device)
        x = function.relu(self.fc1(state))
        x = function.relu(self.fc2(x))
        actions = self.fc3(x)
        
        return actions    

In [5]:
# What makes DQN powerful are the max memory and batch size.
# As the agent plays the game, it stores all its actions, state, 
# reward, new station transitions and done flags in its memory
# This allows it to sample from that memory to collect all of the experiences its used
# to update its values for these actions

#params
# gamma - discount factor of future rewards
# epsilon - how greedy
# eps_end - Final epsilon threshold we want to reach\
# eps_dec - Decrease rate for epsilon

class Agent(object):
    
    def __init__(self, gamma, epsilon, learning_rate, input_dimensions, batch_size, number_actions, 
                     max_memory_size=1000000, eps_end=0.01, eps_dec=0.996):
        self.gamma = gamma
        self.epsilon = epsilon
        self.eps_min = eps_min
        self.learning_rate = learning_rate
        self.input_dimensions = input_dimensions
        self.batch_size = batch_size
        self.number_actions = number_actions
        self.action_space = [i for i in range(number_dactions)]
        self.max_memory_size = max_memory_size
        self.DeepQNetwork(learning_rate, input_dimensions, num_actions=self.number_actions,
                             fc1_dimensions=256, fc2_dimensions=256)
        self.state_memory = np.zeros((self.max_memory_size, *input_dims))
        self.action_memory = np.zeros((self.max_memory_size, self.number_actions), dtype=np.uint8)
        self.reward_memory = np.zeros(self.max_memory_size)
        self.terminal_memory = np.zeros(self.max_memory_size, dtype=np.uint8)
        self.memory_counter = 0
        
        def store_transition(self, state, action, reward, state_, terminal):
            index = self.memory_counter % self.max_memory_size
            self.state_memory[index] = state
            actions = np.zeros(self.number_actions)
            actions[action] = 1.0
            self.action_memory[index] = actions
            self.reward_memory[index] = reward
            self.terminal_memory[index] = 1 - terminal
            self.new_state_memory[index] = state_
            self.memory_counter = self.memory_counter + 1
            
        def choose_action():
            rand = np.random.random()
            if rand < self.epsilon:
                action = np.random.choice(self.action_space)
            else:
                actions = self.Q_eval.forward(observation)
                action = T.argmax(actions).item()
            return action